In [1]:
import re
import os
import sys
import gzip
import pandas as pd

sys.path.append("../")

In [2]:
from io_help import *

In [3]:
data_dir = "/net/dali/home/chikina/shared_data/SELEX"

rounnd_file_names = sorted([x for x in os.listdir(data_dir) if ('.gz' in x and "ZeroCycle" not in x)])

In [4]:
assert len(rounnd_file_names) == 2069

In [5]:
print(f"found {len(rounnd_file_names)} number of files:")
for i in range(0,10):
    print(rounnd_file_names[i])

found 2069 number of files:
ALX3_ESAE_TGCAAG20NGA_1.txt.gz
ALX3_ESAE_TGCAAG20NGA_2.txt.gz
ALX3_ESAE_TGCAAG20NGA_3.txt.gz
ALX3_ESAE_TGCAAG20NGA_4.txt.gz
ALX3_ESZ_TGTAAA20NAAG_1.txt.gz
ALX3_ESZ_TGTAAA20NAAG_2.txt.gz
ALX3_ESZ_TGTAAA20NAAG_3.txt.gz
ALX3_ESZ_TGTAAA20NAAG_4.txt.gz
ALX4_ESW_TGTGTC20NGA_1.txt.gz
ALX4_ESW_TGTGTC20NGA_2.txt.gz


In [6]:
def get_regex_matches(string, protein_id, round_id, verbose = False):
    """
        We are interested only a particular {round_id} so only retrieve those files from the base dir
    """
    
    pattern = rf"^{protein_id}_([A-Za-z]+)_([A-Za-z]+)(\d+N)([A-Z]+)?_{round_id}\.txt\.gz"
    matches = re.findall(pattern, string)
    
    if matches and verbose:
        code1, code2, code3, code4, code5, code6 = matches[0]
        print(code1)  # Output: Alx1
        print(code2)  # Output: ESZ
        print(code3)  # Output: TAAAGC
        print(code4)  # Output: 20N
        print(code5)  # Output: CG

    if not(matches):
        if(verbose):
            print(f"Regex failed at {string}")
        return None 
        
    return matches[0]
    
def additional_check(file_name, regex_out, protein_id):
    codes = file_name.split("_")

    assert regex_out[0]  == codes[1]
    assert protein_id == codes[0]

    return

In [7]:
df = pd.read_csv('protein_info.csv')
all_proteins = set(df['HNGC-name'].tolist())

In [8]:
# Usage
file_name = 'ALX3_ESZ_TGTAAA20NAAG_4.txt.gz'
line_count = count_lines_in_gzfile(f"{data_dir}/{file_name}")

print(f"Number of lines in {file_name}: {line_count}")

Number of lines in ALX3_ESZ_TGTAAA20NAAG_4.txt.gz: 328520


In [9]:
count = count_matching_files('ALX3',4,data_dir)
count

2

In [10]:
def is_valid_dna(seq):
    valid_chars = set('ACGT')
    return all(c in valid_chars for c in seq)

### Load unique dna_fragments in Zero Cycle

In [11]:
unique_zero_cycle_dna_seq = set()

count = 0

zero_cycle_file_names = sorted([x for x in os.listdir(data_dir) if ('.gz' in x and "ZeroCycle" in x)])

for idx, file in enumerate(zero_cycle_file_names):
    if(idx%10==0):
        print(idx, end = ",")
    
    with gzip.open(f"{data_dir}/{zero_cycle_file_names[idx]}", 'rt') as f:
        for i, line in enumerate(f):
            if i % 4 == 1:  # Every fourth line, starting with the second line (index 1)
                temp = line.strip()
                
                # add valids to a 
                if((is_valid_dna(temp))):
                    #zero_cycle_dna_seq.append(temp)
                    unique_zero_cycle_dna_seq.add(temp)
                    count+=1

0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300,310,320,330,340,350,360,370,380,390,400,410,420,430,440,

In [12]:
len(unique_zero_cycle_dna_seq)

286966705

In [17]:
rf_dict

{'1': [], '2': [], '3': [], '4': ['Zic3_ESAF_TCAAAA20NAAA_4.txt.gz'], '5': []}

In [19]:
r_idx = 4

fc = 0

rp_dict    = { '1' : [], '2' : [], '3' : [], '4' : [], '5' : []} # proteins in each round_id
rf_dict    = { '1' : [], '2' : [], '3' : [], '4' : [], '5' : []} # file_name in each round_id
regex_dict = { '1' : [], '2' : [], '3' : [], '4' : [], '5' : []} # the whole segmented file keys

for protein_id in sorted(all_proteins): ## Loop through each protein

    m = 0 
    for string in rounnd_file_names:
        # get file matches for protein & round_id 
        matches = get_regex_matches(string, protein_id, r_idx) 
        
        if(matches != None):
            
            # add protein, file_name, and regex
            rp_dict[str(r_idx)].append(matches[0])
            rf_dict[str(r_idx)].append(string)
            regex_dict[str(r_idx)].append(matches)
            
            # sanity check
            additional_check(string, matches, protein_id)
            m+=1
    
    
    assert count_matching_files(protein_id, r_idx, data_dir) == m
    fc += m
    
print(f"Number of files matched {fc}; all files {len(rounnd_file_names)}")

Number of files matched 517; all files 2069


In [22]:
for string in rounnd_file_names:
    
    if ('_4.txt.gz' in string) and string not in rf_dict['4']:
        print(string)

Egr1_ESP_TGCGCC20NCG_4.txt.gz


In [ ]:
Egr1_ESP_TGCGCC20NCG_4

In [25]:
count_lines_in_gzfile(f"{data_dir}/Egr1_ESP_TGCGCC20NCG_4.txt.gz")//4

2995744

In [ ]:
## Dictionaries to store 
# csv_file = open('summary.csv', 'w')
# csv_file.write(f"protein_id,all_count,valid_count,total_unique_count,intersection_count\n")

r_idx = 4

protein_names = []

protein_all_counts    = [] ## can include Ns
protein_valid_counts  = [] ## all validss
protein_unique_counts = []

global_count = 0
global_valid_count = 0

all_round_four_seq = set()

for protein_id in sorted(all_proteins): ## Loop through each protein

    rp_dict    = { '1' : [], '2' : [], '3' : [], '4' : [], '5' : []} # proteins in each round_id
    rf_dict    = { '1' : [], '2' : [], '3' : [], '4' : [], '5' : []} # file_name in each round_id
    regex_dict = { '1' : [], '2' : [], '3' : [], '4' : [], '5' : []} # the whole segmented file keys
    
    for string in rounnd_file_names:
        # get file matches for protein & round_id 
        matches = get_regex_matches(string, protein_id, r_idx) 
        
        if(matches != None):
            
            # add protein, file_name, and regex
            rp_dict[str(r_idx)].append(matches[0])
            rf_dict[str(r_idx)].append(string)
            regex_dict[str(r_idx)].append(matches)
            
            # sanity check
            additional_check(string, matches, protein_id)

    
    filtered_file_names = rf_dict[str(r_idx)] ##filtered file names for protein and round of interest
    
    assert count_matching_files(protein_id, r_idx, data_dir) == len(filtered_file_names)
        
    all_count   = 0
    valid_count = 0
    count2 = 0
    
    dna_seq = []
    
    ## loop through each relevant file 
    for file in filtered_file_names:
        
        ## count number of lines through bash
        count2 += count_lines_in_gzfile(f"{data_dir}/{file}")//4
        
        with gzip.open(f"{data_dir}/{file}", 'rt') as f:
            for i, line in enumerate(f):
                if i % 4 == 1:  # Every fourth line, starting with the second line (index 1)
                    temp = line.strip()
                    
                    ## check if there's any 'N's in the sequence
                    if(is_valid_dna(temp)):
                        dna_seq.append(temp)
                        all_round_four_seq.add(temp)
                        valid_count += 1
                    
                    all_count += 1
    
    assert all_count == count2
    
    global_count       += all_count
    global_valid_count += valid_count

    unique_dna_seq = set(dna_seq)

    total_unique_count = len(unique_dna_seq)
    # print(total_count)

    protein_names.append(protein_id)
    
    # append counts
    protein_all_counts.append(all_count)
    protein_valid_counts.append(valid_count)
    protein_unique_counts.append(total_unique_count)
    
    intersection_count = sum(1 for frag in unique_dna_seq if frag in unique_zero_cycle_dna_seq)
    
    #csv_file.write(f"{protein_id},{all_count},{valid_count},{total_unique_count},{intersection_count}\n")
    print(f"{protein_id:>10} {all_count:>10} {valid_count:>10} {total_unique_count:>10} {intersection_count:>10}")

      ALX3     336781     336701     151063         67
      ALX4    5159539    5159290    1946739        898
        AR    2250446    2249546    1116328        622
     ARNTL    2021565    2021452    1557409        822
       ARX    1324032    1323723    1109695        679
      ATF4    2687716    2687502    1992452       1003
      ATF7    2094093    2094090    1688239          2
      Alx1    2648368    2647764     608987        303
      Alx4    3558192    3557893    1366067        953
        Ar    4636886    4636514    2464167       1834
       Arx     333527     327442     294700        151
     Ascl2     425545     425441     164340        148
      Atf4      28871      28862       3901          0
     Atoh1    1601470    1601106     727491        349
    BARHL2    1907015    1906268    1348666        218
     BARX1     174399     174320     135524         63
     BATF3     482352     473579     445794        259
     BCL6B     787864     783415     519273        336
   BHLHA15

In [ ]:
# print("redda")

In [ ]:
# csv_file.close()

In [15]:
len(all_round_four_seq)

392671130

In [ ]:
# protein_names

In [26]:
count_lines_in_gzfile(f"{data_dir}/Egr1_ESP_TGCGCC20NCG_4.txt.gz")//4

2995744

In [27]:
## This protien is not in the all protein list file. so go through that manually

In [28]:
with gzip.open(f"{data_dir}/Egr1_ESP_TGCGCC20NCG_4.txt.gz", 'rt') as f:
    for i, line in enumerate(f):
        if i % 4 == 1:  # Every fourth line, starting with the second line (index 1)
            temp = line.strip()

            ## check if there's any 'N's in the sequence
            if(is_valid_dna(temp)):
                all_round_four_seq.add(temp)

In [29]:
len(all_round_four_seq)

394325561